In [1]:
import pandas as pd
import numpy as np

# -----------------------------
# Core Sales Dataset
# -----------------------------
data = {
    "date": pd.date_range(start="2024-01-01", periods=180, freq="D"),
    "genre": ["Sci-Fi"] * 90 + ["Fantasy"] * 90,
    "platform": np.random.choice(
        ["Amazon KDP", "Barnes & Noble", "IngramSpark", "Apple Books"],
        size=180,
        p=[0.55, 0.20, 0.15, 0.10]
    ),
    "format": np.random.choice(
        ["Ebook", "Paperback", "Hardcover", "Audiobook"],
        size=180,
        p=[0.45, 0.30, 0.15, 0.10]
    ),
    "units_sold": np.random.poisson(lam=18, size=180),
    "price_per_unit": np.random.choice(
        [3.99, 4.99, 9.99, 14.99, 19.99],
        size=180
    ),
    "platform_fee_pct": np.random.choice(
        [0.30, 0.40, 0.55],  # typical ebook vs print/audiobook cuts
        size=180
    ),
    "amazon_sales_rank": np.random.randint(5_000, 250_000, size=180),
    "page_views": np.random.randint(150, 2500, size=180),
    "ad_spend_usd": np.random.choice([0, 5, 10, 20, 50], size=180),
    "review_count": np.random.poisson(lam=2, size=180),
    "average_rating": np.round(np.random.uniform(3.5, 4.8, size=180), 2),
    "inventory_print": np.random.randint(50, 600, size=180)
}

df = pd.DataFrame(data)


In [2]:
# Gross Revenue
df["gross_revenue"] = df["units_sold"] * df["price_per_unit"]

# Net Royalties (after platform fees)
df["royalty_earned"] = df["gross_revenue"] * (1 - df["platform_fee_pct"])

# Sales Velocity (units per day proxy)
df["sales_velocity"] = df["units_sold"]


In [3]:
df["month"] = df["date"].dt.month
df["week"] = df["date"].dt.isocalendar().week

monthly_trends = df.groupby(["genre", "month"])[
    ["units_sold", "gross_revenue", "royalty_earned"]
].sum().reset_index()

monthly_trends.head()


,genre,month,units_sold,gross_revenue,royalty_earned
0,Fantasy,3,15,224.85,134.910
1,Fantasy,4,562,6823.38,3910.229
2,Fantasy,5,528,6152.72,3493.136
3,Fantasy,6,536,5555.64,3390.949
4,Sci-Fi,1,588,7284.12,4270.734


In [4]:
competitors = pd.DataFrame({
    "title": [
        "Starfall Armada", "Void Empire", "Dragonfire Oath",
        "The Iron Mage", "Nebula Rising", "Crown of Ashes"
    ],
    "genre": ["Sci-Fi", "Sci-Fi", "Fantasy", "Fantasy", "Sci-Fi", "Fantasy"],
    "avg_price": [4.99, 6.99, 5.99, 7.99, 3.99, 6.49],
    "amazon_rank": [12000, 8500, 6700, 4300, 25000, 3900],
    "review_count": [850, 1200, 2200, 3100, 400, 2800],
    "avg_rating": [4.4, 4.6, 4.7, 4.8, 4.1, 4.7]
})

competitors


,title,genre,avg_price,amazon_rank,review_count,avg_rating
0,Starfall Armada,Sci-Fi,4.99,12000,850,4.4
1,Void Empire,Sci-Fi,6.99,8500,1200,4.6
2,Dragonfire Oath,Fantasy,5.99,6700,2200,4.7
3,The Iron Mage,Fantasy,7.99,4300,3100,4.8
4,Nebula Rising,Sci-Fi,3.99,25000,400,4.1
5,Crown of Ashes,Fantasy,6.49,3900,2800,4.7


In [5]:
platform_format_perf = df.groupby(
    ["platform", "format"]
)[["units_sold", "gross_revenue", "royalty_earned"]].sum().reset_index()

platform_format_perf.sort_values("units_sold", ascending=False).head()


,platform,format,units_sold,gross_revenue,royalty_earned
1,Amazon KDP,Ebook,883,10740.17,6097.9080
2,Amazon KDP,Hardcover,459,4493.41,2595.9605
3,Amazon KDP,Paperback,387,3975.13,2203.3975
9,Barnes & Noble,Ebook,262,2752.38,1636.6850
11,Barnes & Noble,Paperback,235,2490.65,1381.2060


In [7]:
print_sales = df.loc[
    df["format"].isin(["Paperback", "Hardcover"])
].copy()

print_sales["sell_through_rate"] = (
    print_sales["units_sold"] /
    (print_sales["units_sold"] + print_sales["inventory_print"])
)

print_sales[["platform", "format", "sell_through_rate"]].head()


,platform,format,sell_through_rate
1,Amazon KDP,Hardcover,0.028120
2,Amazon KDP,Hardcover,0.031311
3,IngramSpark,Paperback,0.040254
4,IngramSpark,Hardcover,0.045635
5,Amazon KDP,Hardcover,0.304878


In [8]:
df["conversion_rate"] = df["units_sold"] / df["page_views"]

df["marketing_roi"] = np.where(
    df["ad_spend_usd"] > 0,
    df["royalty_earned"] / df["ad_spend_usd"],
    np.nan
)

df[["conversion_rate", "marketing_roi"]].describe()


,conversion_rate,marketing_roi
count,180.000000,142.000000
mean,0.022000,10.223097
std,0.021273,9.634024
min,0.002817,0.449100
25%,0.009142,3.514280
50%,0.014448,7.484330
75%,0.026842,13.943025
max,0.163522,69.965000


In [9]:
df["year"] = df["date"].dt.year

yoy = df.groupby(["genre", "year"])[
    ["units_sold", "gross_revenue", "royalty_earned"]
].sum().reset_index()

yoy


,genre,year,units_sold,gross_revenue,royalty_earned
0,Fantasy,2024,1641,18756.59,10929.224
1,Sci-Fi,2024,1625,17669.75,10230.913


In [10]:
dashboard = df.groupby(
    ["genre", "platform", "format"]
).agg(
    units_sold=("units_sold", "sum"),
    revenue=("gross_revenue", "sum"),
    royalties=("royalty_earned", "sum"),
    avg_conversion=("conversion_rate", "mean"),
    avg_rating=("average_rating", "mean")
).reset_index()

dashboard.head()


,genre,platform,format,units_sold,revenue,royalties,avg_conversion,avg_rating
0,Fantasy,Amazon KDP,Audiobook,10,99.90,59.940,0.006064,4.100000
1,Fantasy,Amazon KDP,Ebook,485,6100.15,3485.707,0.023493,4.074615
2,Fantasy,Amazon KDP,Hardcover,230,2183.70,1212.862,0.026154,4.110909
3,Fantasy,Amazon KDP,Paperback,238,2243.62,1246.612,0.023488,3.965000
4,Fantasy,Apple Books,Audiobook,20,99.80,44.910,0.008897,3.640000
